# Imports

In [1]:
from train.utils import *
import os
import shutil
from PIL import Image
from pytorch_fid import fid_score
import copy

import torch.distributions as D

import torch.optim.lr_scheduler as lrsc
import wandb
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

model_dict = dict()
model_dict['nf'] = 128
model_dict['ch_mult'] = (1, 2, 2, 2)
model_dict['num_res_blocks'] = 4
model_dict['attn_resolutions'] = (16,)
model_dict['dropout'] = 0.1
model_dict['resamp_with_conv'] = True
model_dict['conditional'] = True
model_dict['nonlinearity'] = 'swish'
model_dict['sigma_max'] = 50
model_dict['sigma_min'] = 0.01
model_dict['num_scales'] = 1000
data_dict = dict()
data_dict['image_size'] = 32
data_dict['num_channels'] = 3
data_dict['centered'] = True
config_dict = dict()
config_dict['model'] = dotdict(model_dict)
config_dict['data'] = dotdict(data_dict)
config = dotdict(config_dict)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wandb.login()
train_loader, val_loader = get_datasets()

wandb: Currently logged in as: vvoganesyan (use `wandb login --relogin` to force relogin)


Files already downloaded and verified
Files already downloaded and verified


In [2]:
# config.data.image_size = 16

# Sampling

In [3]:
eps_th = Unet(config)
eps_th.to(device)

dequant = GaussDequant(scale=0)
dequant.to(device)
ema_ = ema.ExponentialMovingAverage(eps_th.parameters(), decay=0.9999)

In [4]:
exp_name = 'ddpm'
eps_th.load_state_dict(torch.load(f'{exp_name}_model'))
ema_.load_state_dict(torch.load(f'{exp_name}_ema'))

In [5]:
ema_.copy_to(eps_th.parameters())

In [94]:
# config.data.image_size = 32
# config.model.ch_mult = (1,2,2,2)
# eps_th_big = Unet(config)
# eps_th_big.to(device)

# ema_ = ema.ExponentialMovingAverage(eps_th_big.parameters(), decay=0.9999)

In [95]:
# eps_th_big.load_state_dict(torch.load('ddpm_model'))
# ema_.load_state_dict(torch.load('ddpm_ema'))

In [96]:
# ema_.copy_to(eps_th_big.parameters())

In [8]:
# with torch.no_grad():
#     save_callable(lambda: gen_discrete(eps_th), "data_2_Grisha", 128*128)

In [68]:
# cov_matrix = torch.tensor([[1.0084, 0.9407, 0.9244, 0.8750],
#         [0.9407, 1.0070, 0.8756, 0.9229],
#         [0.9244, 0.8756, 0.9955, 0.9278],
#         [0.8750, 0.9229, 0.9278, 0.9940]])

# Two models

In [82]:
alpha02 = alpha(torch.tensor(0.2))
sigma02 = sigma_2(torch.tensor(0.2))
x = upsample_image(lowres1[0], cov_matrix=cov_matrix, alpha_t=alpha02, sigma_t_2=sigma02)

<ipython-input-67-021575f8bc35>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_cov = torch.tensor(initial_cov, device=device, dtype=torch.float)


In [105]:
num = 0
with torch.no_grad():
    for i in tqdm(range(40)):
        x = upsample_image(lowres1[i], cov_matrix=cov_matrix, alpha_t=alpha02, sigma_t_2=sigma02)
        y = sample_sde_mid(x, ts=0.2, tf=0.0)
        num = save_batch(y, 'data_2_us', num)

<ipython-input-67-021575f8bc35>:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_cov = torch.tensor(initial_cov, device=device, dtype=torch.float)
